In [ ]:
from huggingface_hub import login
import os

# Login to Hugging Face
login("MY_TOKEN") #hidden for security

/home/sagemaker-user/.conda/envs/llama_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForCausalLM
import torch
import json

In [5]:
# import requests

# model_url = "https://huggingface.co/meta-llama/Llama-3.1-8B"

# try:
#     response = requests.get(model_url)
#     if response.status_code == 200:
#         print("Successfully accessed the model repository.")
#     else:
#         print(f"Failed to access model repository. Status code: {response.status_code}")
# except Exception as e:
#     print(f"Error accessing model repository: {e}")

Successfully accessed the model repository.


In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "meta-llama/Llama-3.1-8B"

# # Optionally, specify a cache directory
# cache_dir = "./cache_llama_models"

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir=cache_dir)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
#     trust_remote_code=True,
#     cache_dir=cache_dir
# )

Loading checkpoint shards: 100%|██████████| 4/4 [02:02<00:00, 30.57s/it]


In [4]:
from huggingface_hub import login
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFacePipeline



In [1]:
HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
import torch
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM

# Paths
model_path = "/home/sagemaker-user/Llama-3.1-8B/original/"  # Path to consolidated.pth
checkpoint_path = f"{model_path}/consolidated.pth"  # Full path to the weights
tokenizer_id = "meta-llama/Llama-3.1-8B"  # Hugging Face model ID for tokenizer

# Load tokenizer and config from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
config = AutoConfig.from_pretrained(tokenizer_id)

# Initialize the model with Hugging Face config
model = LlamaForCausalLM(config)

# Load weights from consolidated.pth
state_dict = torch.load(checkpoint_path, map_location="cpu")
model.load_state_dict(state_dict, strict=False)

# Move the model to the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Set up a text-generation pipeline
from transformers import pipeline

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Generate text
output = text_gen("Hey, how are you doing today?", max_length=50)
print(output)




In [5]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
import torch

# Define model details
model_name = "meta-llama/Llama-3.1-8B"

# Specify an absolute where llama
cache_dir = "./cache_llama_models"

# Ensure the cache directory exists
os.makedirs(cache_dir, exist_ok=True)



# Ensure the cache directory exists
os.makedirs(cache_dir, exist_ok=True)

model = load_checkpoint_and_dispatch(
    model,
    checkpoint=checkpoint_dir,  # Use the correct directory containing the model weights
    device_map="auto",          # Automatically maps model parts to CPU/GPU  # Specify folder for offloading     # Enable state dict offloading
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    cache_dir=cache_dir,
    padding_side="right",  # Ensure padding on the correct side
    truncation=True        # Explicitly truncate inputs
)

# Example pipeline usage (optional)
pipeline_example = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  7.88it/s]


ValueError: `checkpoint` should be the path to a file containing a whole state dict, or the index of a sharded checkpoint, or a folder containing a sharded checkpoint or the whole state dict, but got ./cache_llama_models/Llama-3.1-8B.

In [ ]:
# Create a text-generation pipeline with adjusted parameters
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a flight from one location to another and asks about a specific day, with additional information sometimes added like number of passengers, class, if only non-stop, etc. This is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights"].
   - The "action_input" field contains:
     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.
     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields should be included if mentioned and returned as an empty string if not mentioned:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer for the number of children.
       - "infants": An integer for the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
     - "max": Always set to 5.

3. Generate one example per response object. Be variable in the input format and the output fields following the guidelines above.
"""

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the system prompt
    response = pipeline(
        SYSTEM_PROMPT,
        max_length=512,
        temperature=temperature,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Extract the generated text
    generated_text = response[0]["generated_text"]

    try:
        # Parse the response and append it to the examples list
        example = json.loads(generated_text)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {generated_text}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

In [ ]:
import boto3
import json

# SageMaker endpoint details
endpoint_name = "jumpstart-dft-llama-3-2-1b-instruct-20241204-042357"
region = "us-east-1"

input = """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a flight from one location to another and asks about a specific day, with additional information sometimes added like number of passengers, class, if only non-stop, etc. This is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights"].
   - The "action_input" field contains:
     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.
     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields should be included if mentioned and returned as an empty string if not mentioned:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer for the number of children.
       - "infants": An integer for the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
     - "max": Always set to 5.

3. Generate one example per response object. Be variable in the input format and the output fields following the guidelines above.
"""

# Input data for the model
payload = {
    "inputs": """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a flight from one location to another and asks about a specific day, with additional information sometimes added like number of passengers, class, if only non-stop, etc. This is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights"].
   - The "action_input" field contains:
     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.
     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields should be included if mentioned and returned as an empty string if not mentioned:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer for the number of children.
       - "infants": An integer for the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
     - "max": Always set to 5.

3. Generate one example per response object. Be variable in the input format and the output fields following the guidelines above.
""",
    "parameters": {
        "max_length": 50,
        "temperature": 1,
        "stream": "false"
    }
}

# Initialize the SageMaker runtime client
sagemaker_client = boto3.client("sagemaker-runtime", region_name=region)


examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the system prompt
    response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json"
)

    # Extract the generated text
    generated_text = response["Body"].read().decode("utf-8")

    try:
        # Parse the response and append it to the examples list
        example = generated_text
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {generated_text}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(f"{examples[idx]}")
# Make the request to the endpoint

## Start

In [64]:
import transformers
import torch
import json
import re

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )

# Use a different variable name to avoid confusion
chat_pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


Loading checkpoint shards: 100%|██████████| 4/4 [01:57<00:00, 29.45s/it]


In [6]:
# llm = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto"
# )

Loading checkpoint shards: 100%|██████████| 4/4 [00:38<00:00,  9.53s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [65]:
messages = [
    {"role": "system", "content": "You create 1 data point in the format and structure asked by the user."},
    {"role": "user", "content": """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a flight from one location to another and asks about a specific day, with additional information sometimes added like number of passengers, class, if only non-stop, etc. This is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights"].
   - The "action_input" field contains:
     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.
     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields should be included if mentioned and returned as an empty string if not mentioned:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer for the number of children.
       - "infants": An integer for the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
     - "max": Always set to 5.
"""}
]

# Generate output using the chat pipeline
outputs = chat_pipeline(
    messages,
    max_new_tokens=256,
)

print(outputs)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'system', 'content': 'You create 1 data point in the format and structure asked by the user.'}, {'role': 'user', 'content': '\nGenerate a JSON object where:\n1. "input" is a single string where a user requests information for a flight from one location to another and asks about a specific day, with additional information sometimes added like number of passengers, class, if only non-stop, etc. This is provided in plain text, not JSON.\n2. "output" contains the corresponding Amadeus API call in JSON format:\n   - The "action" field is always ["search_flights"].\n   - The "action_input" field contains:\n     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.\n     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.\n     - "departure_date": A date in "YYYY-MM-DD" format.\n     - "adults": An integer representing the number of adults (default is 1).\n     - Option

In [66]:
# Store generated examples
examples = []

# Loop to generate 10 examples
for i in range(10):
    try:
        # Call the pipeline with the defined messages as a positional argument
        outputs = chat_pipeline(
            messages,
            max_new_tokens=256,
        )

        # Extract the assistant's content from the outputs
        assistant_content = outputs[0]["generated_text"][-1]['content']

        # Extract JSON block corresponding to assistant content
        json_match = re.search(r"json\n(.*?)\n", assistant_content, re.DOTALL)
        if json_match:
            json_text = json_match.group(1)
            # Parse JSON and append to examples
            example = json.loads(json_text)
            examples.append(example)
        else:
            print(f"No valid JSON found at iteration {i + 1}")

    except json.JSONDecodeError:
        print(f"Failed to decode JSON at iteration {i + 1}: {assistant_content}")
    except Exception as e:
        print(f"Error during generation at iteration {i + 1}: {str(e)}")

# Pretty-print all valid JSON examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON at iteration 1: ```json
{
  "input": "Flight from LAX to JFK on 2024-07-27 for 2 adults and 1 child in economy class with non-stop flights only",
  "output": {
    "action": ["search_flights"],
    "action_input": {
      "origin": "LAX",
      "destination": "JFK",
      "departure_date": "2024-07-27",
      "adults": "2",
      "children": "1",
      "class": "ECONOMY",
      "nonStop": "true",
      "max": "5"
    }
  }
}
```

This JSON object represents a user's request for flights from LAX to JFK on July 27, 2024, for 2 adults and 1 child in economy class with non-stop flights only. The corresponding Amadeus API call is also provided in JSON format.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON at iteration 2: Here's a Python script to generate the JSON object based on the user's input:

```python
import json
import re

def generate_amadeus_api_call(input_str):
    # Regular expression to extract information from the input string
    pattern = r"from\s+([A-Z]{3})\s+to\s+([A-Z]{3})\s+on\s+([A-Za-z]{3}\s+\d{1,2},\s\d{4})\s*(.*)"
    match = re.match(pattern, input_str)
    
    if match:
        origin = match.group(1)
        destination = match.group(2)
        departure_date = match.group(3).replace(",", "").replace(" ", "-")
        additional_info = match.group(4)
        
        adults = 1
        return_date = ""
        children = ""
        infants = ""
        class_ = ""
        nonStop = ""
        maxPrice = ""
        
        # Parse additional information
        if additional_info:
            words = additional_info.split()
            for word in words:
                if word.isdigit():
                    adults = int(word)
          

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Failed to decode JSON at iteration 7: Here's a Python function to generate the JSON object based on the user's input:

```python
import json

def generate_amadeus_api_call(input_string):
    # Parse the input string
    words = input_string.split()
    origin = None
    destination = None
    departure_date = None
    return_date = None
    adults = 1
    children = 0
    infants = 0
    class_type = None
    non_stop = False
    max_price = None

    for i, word in enumerate(words):
        if word.upper() in ["FROM", "FROMTO"]:
            origin = words[i+1][:3].upper()
        elif word.upper() in ["TO", "ARRIVAL"]:
            destination = words[i+1][:3].upper()
        elif word.upper() in ["ON", "DATE"]:
            departure_date = words[i+1]
        elif word.upper() in ["RETURN", "BACK"]:
            return_date = words[i+1]
        elif word.upper() == "ADULTS":
            adults = int(words[i+1])
        elif word.upper() == "CHILDREN":
            children = int(words[i+

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON at iteration 8: Here is an example JSON object based on your request:

```json
{
  "input": "Flight from LAX to JFK on 2024-07-25 for 2 adults and 1 child, non-stop, economy class",
  "output": {
    "action": ["search_flights"],
    "action_input": {
      "origin": "LAX",
      "destination": "JFK",
      "departure_date": "2024-07-25",
      "adults": 2,
      "children": 1,
      "class": "ECONOMY",
      "nonStop": true,
      "max": 5
    }
  }
}
```

Here's another example with a return date and a maximum price:

```json
{
  "input": "Flight from LAX to JFK on 2024-07-25 and return on 2024-07-30 for 2 adults, non-stop, economy class with a max price of 500",
  "output": {
    "action": ["search_flights"],
    "action_input": {
      "origin": "LAX",
      "destination": "JFK",
      "departure_date": "2024-07-25


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON at iteration 9: Here's an example JSON object based on the input string "Flight from New York (JFK) to Los Angeles (LAX) on 2024-07-25, non-stop, economy class for 2 adults and 1 child".

```json
{
  "input": "Flight from New York (JFK) to Los Angeles (LAX) on 2024-07-25, non-stop, economy class for 2 adults and 1 child",
  "output": {
    "action": ["search_flights"],
    "action_input": {
      "origin": "JFK",
      "destination": "LAX",
      "departure_date": "2024-07-25",
      "adults": "2",
      "class": "ECONOMY",
      "children": "1",
      "nonStop": "true",
      "max": "5"
    }
  }
}
```

Here's another example based on the input string "Flight from Paris (CDG) to Tokyo (NRT) on 2024-08-01 for 3 adults with a maximum price of 1000 USD".

```json
{
  "input": "Flight from Paris (CDG) to Tokyo
Failed to decode JSON at iteration 10: Here's a Python script to generate the JSON object based on the user's input:

```python
import json

def generate_amade